### 펀다 상점 매출 Using Facebook Prophet 
- 대회 예측용 코드 
- 카드거래 취소 적용한 data_pos.csv 사용
- 파라미터 
    - 샘플링 관련
        - sampling_period_days : 14
        - make_future : (periods=6,freq='2W')
    - 트랜드 관련
        - changepoint_range : 기본 0.8
        - changepoint_prior_scale : cps = max(0.5*((1 - (count_zero)/len(store_ts))),0.05)

In [1]:
import numpy as np
import pandas as pd
from pandas import datetime
import matplotlib.pyplot as plt
from fbprophet import Prophet
import warnings
warnings.filterwarnings(action='ignore')

from datetime import date, timedelta
from lunarcalendar import Lunar, Converter
print("done")

done


In [2]:
df = pd.read_csv('data_pos.csv')
print("done")

done


In [3]:
df.transacted_date = pd.to_datetime(df.transacted_date)
test_groupby_date_store = df.groupby(['transacted_date', 'store_id'])['amount'].sum()
test_groupby_date_store = test_groupby_date_store.reset_index()
test_groupby_date_store = test_groupby_date_store.set_index('transacted_date')
store_list = test_groupby_date_store.store_id.unique()
store_list.sort()
print("store list :")
print(store_list)
ts_train = test_groupby_date_store
print("done")

store list :
[   0    1    2 ... 2134 2135 2136]
done


In [4]:
def sampling(input_df, sampling_period_days):
    input_df = input_df[len(input_df) % sampling_period_days:].resample(str(sampling_period_days) + 'D').sum()
    return input_df

In [ ]:
store_amount = {}
store_amount['store_id'] = []
store_amount['amount'] = []
i = 0
total = len(store_list)
for store_id in store_list[:]:
    i += 1
    
    store_ts = ts_train[ts_train.store_id==store_id]
    store_ts = sampling(store_ts,14)
    count_zero = len(store_ts[(store_ts.amount == 0) | (store_ts.amount.isna())])
    cps = max(0.5*((1 - (count_zero)/len(store_ts))),0.05)
    print("{}/{} cps: {}".format(i,total,cps))
    store_ts['y'] = store_ts['amount']
    store_ts['ds'] = store_ts.index
    store_ts = store_ts.drop('amount', axis=1)
    store_ts = store_ts.drop('store_id', axis=1) 
    m = Prophet(changepoint_prior_scale=cps)
    m_fit = m.fit(store_ts)
    future = m_fit.make_future_dataframe(periods=6,freq='2W')
    forecast = m_fit.predict(future)
    size = store_ts.shape[0]
    result = np.sum(forecast.yhat[size:])
    store_amount['store_id'].append(store_id)
    store_amount['amount'].append(result)
    
#     print(forecast[size:])
#     print(forecast.yhat[size:])
#     fig1 = m.plot(forecast)

In [8]:
out_ff = pd.DataFrame(store_amount,columns=['store_id','amount'])  
print(out_ff)
out_ff.to_csv("prophet2W6P-CPS0.5.csv", mode='w',index=False) # 새파일로 덮어쓰기

      store_id        amount
0            0  1.633533e+06
1            1  2.785406e+05
2            2  1.137867e+06
3            4  2.398051e+06
4            5  7.192685e+05
5            6  4.894252e+06
6            7 -1.564414e+05
7            8  4.152284e+06
8            9  1.525754e+06
9           10  1.712936e+06
10          11  1.275503e+06
11          12  1.116099e+06
12          13  1.770893e+06
13          14  9.552226e+06
14          15  1.272486e+06
15          16  4.485323e+05
16          17  5.940596e+05
17          18  9.875865e+05
18          19  4.946695e+06
19          20  5.386338e+06
20          22  5.849912e+05
21          23  2.374524e+06
22          24  2.833588e+06
23          25  4.282547e+05
24          26  4.287284e+05
25          27  6.405410e+06
26          28  3.096619e+06
27          29  1.764584e+06
28          30  9.580443e+05
29          31  1.821196e+06
...        ...           ...
1937      2106  3.596804e+06
1938      2107  2.583276e+05
1939      2108

In [ ]:
true_sum

In [ ]:
new_true_dict = {}
new_true_dict['store_id'] = []
new_true_dict['amount'] = []
new_test_dict = {}
new_test_dict['store_id'] = []
new_test_dict['amount'] = []
# print(store_list)
for store_id in store_list[:]:
#     print(store_id)
    new_true_dict['amount'].append(true_sum[store_id])
    new_true_dict['store_id'].append(store_id)
    try:
        new_test_dict['amount'].append(test_sum[store_id])
    except:
        print(store_id)
    new_test_dict['store_id'].append(store_id)
# print(new_test_dict)
true_frame = pd.DataFrame(new_true_dict)   
true_frame =true_frame.set_index("store_id")
predict_frame = pd.DataFrame(new_test_dict)
predict_frame = predict_frame.set_index("store_id")
predict_frame = predict_frame['amount'].apply((lambda x: x*1))
print("done")

In [ ]:
plt.figure(figsize=(22,10))
plt.plot(true_frame,label = "true_sum")
plt.plot(predict_frame,label = "predict_sum")
plt.title("Time Series Forecast")
plt.xlabel("store id")
plt.ylabel("total amount")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# MAE 값
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(predict_frame[:114],true_frame[:114])
print(mae)